In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

path = 'return-data/'
filelist = os.listdir(path) 
dfs = [pd.read_json(path+file, lines=True) for file in filelist]
toydata = pd.concat(dfs, ignore_index=True)

In [2]:
toydata.head()

,basket,returnLabel,totalAmount,transactionId,zipCode
0,"[1, 1, 0, 2, 5, 1, 2, 2]",0,264,5221131913,2321
1,"[1, 4, 4, 3, 2, 4]",1,60,4768566069,7597
2,[5],0,60,4791292369,1176
3,"[0, 4, 3]",0,333,4672834572,3087
4,"[2, 3, 1, 2, 0, 0]",0,378,8750635039,9400


In [3]:
toydata = toydata.merge(toydata['basket'].apply(lambda x: pd.Series({'c_0': x.count(0), 'c_1': x.count(1),'c_2': x.count(2),'c_3': x.count(3),'c_4': x.count(4),'c_5': x.count(5)})), left_index=True, right_index=True)

In [4]:
zipcode_one_hot = pd.get_dummies(toydata['zipCode'])

toydata.drop(['totalAmount','basket','transactionId','zipCode'],axis=1,inplace=True)
toydata = pd.concat([toydata,zipcode_one_hot],axis=1)
toydata.head()

,returnLabel,c_0,c_1,c_2,c_3,c_4,c_5,1000,1001,1002,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
0,0,1,3,3,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,1,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,1,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(toydata.drop('returnLabel',axis=1),toydata['returnLabel'], test_size=0.30,random_state=101)

In [6]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(solver='lbfgs')
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [7]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
print(classification_report(y_test,predictions))
print("Accuracy:",accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      9001
           1       0.74      0.19      0.30       299

   micro avg       0.97      0.97      0.97      9300
   macro avg       0.86      0.59      0.64      9300
weighted avg       0.97      0.97      0.96      9300

('Accuracy:', 0.9717204301075268)
